**Imports**

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import time
import import_ipynb
import UTILS as utils
import os
from itertools import repeat
import re
import plotly.express as px
from statistics import mean

import FundamentalAnalysis as fa

importing Jupyter notebook from UTILS.ipynb


In [8]:
api_key = "Here would go my personal API key"

###BEGINNING###

**Generic functions**

In [9]:
def natural_key(string_):
    return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string_) if s]

In [10]:
def read_pickle_file(file_name):
    with open(file_name,'rb') as file:
        raw_data = file.read()
        return pickle.loads(raw_data)

In [11]:
def get_method_files(method, string_distinction):
    outputs_folders = [folder for folder in list(os.listdir(f'models_outputs/{method}_output')) if folder[0].isdigit()]
    rev_sorted_folders = sorted(outputs_folders, key=natural_key, reverse=True)
    outputs_sorted_by_years = dict()
    for folder in rev_sorted_folders:
        folder_output=os.listdir(f'models_outputs/{method}_output/{folder}')
        if len(folder_output):
            pickle_files = list(filter(lambda x: string_distinction in x, folder_output))
            if len(pickle_files):
                outputs_sorted_by_years[natural_key(folder)[0]]=read_pickle_file(f'models_outputs/{method}_output/{folder}/{pickle_files[0]}')
    return outputs_sorted_by_years

In [12]:
def to_csv(sector, sector_df, method):
    sector_df.to_csv(f'models_outputs/{method}_output/sectors/{method}_{sector}')

In [13]:
def get_evolution_per_sector_sorted(years_output, method):
    evolution_per_sector_sorted = dict()
    for sector in years_output[0].keys():
        evolution_per_sector_sorted[sector] = dict()
        for year_disp in years_output.keys():
            evolution_per_sector_sorted[sector][year_disp] = dict()
            if method in ['DCF', 'EPS', 'NCAVPS', 'GGM']:
                nulls_removed_dict = {key:value for key, value in years_output[year_disp][sector].items() if not np.isnan(years_output[year_disp][sector][key][0])}
                for key, value in sorted(nulls_removed_dict.items(), key=lambda e: e[1][1], reverse=True):
                    evolution_per_sector_sorted[sector][year_disp][key] = value
            elif method == 'piotroskiFscore':
                nulls_removed_dict = {key:value for key, value in years_output[year_disp][sector].items() if not np.isnan(years_output[year_disp][sector][key])}
                for key, value in sorted(nulls_removed_dict.items(), key=lambda e: e[1], reverse=True):
                    evolution_per_sector_sorted[sector][year_disp][key] = value
            else:
                nulls_removed_dict = {key:value for key, value in years_output[year_disp][sector].items() if not np.isnan(years_output[year_disp][sector][key])}
                for key, value in sorted(nulls_removed_dict.items(), key=lambda e: e[1], reverse=False):
                    evolution_per_sector_sorted[sector][year_disp][key] = value
            
    return evolution_per_sector_sorted

In [14]:
def get_evolution_per_year_sorted(years_output, method):
    evolution_per_year_sorted = dict()
    for year_disp in years_output.keys():
        evolution_per_year_sorted[year_disp] = dict()
        for sector in years_output[year_disp].keys():
            evolution_per_year_sorted[year_disp][sector] = dict()
            
            if method in ['DCF', 'EPS', 'NCAVPS', 'GGM']:
                nulls_removed_dict = {key:value for key, value in years_output[year_disp][sector].items() if not np.isnan(years_output[year_disp][sector][key][0])}
                for key, value in sorted(nulls_removed_dict.items(), key=lambda e: e[1][1], reverse=True):
                    evolution_per_year_sorted[year_disp][sector][key] = value
            elif method == 'piotroskiFscore':
                nulls_removed_dict = {key:value for key, value in years_output[year_disp][sector].items() if not np.isnan(years_output[year_disp][sector][key])}
                for key, value in sorted(nulls_removed_dict.items(), key=lambda e: e[1], reverse=True):
                    evolution_per_year_sorted[year_disp][sector][key] = value
            else:
                nulls_removed_dict = {key:value for key, value in years_output[year_disp][sector].items() if not np.isnan(years_output[year_disp][sector][key])}
                for key, value in sorted(nulls_removed_dict.items(), key=lambda e: e[1], reverse=False):
                    evolution_per_year_sorted[year_disp][sector][key] = value
            
    return evolution_per_year_sorted

In [15]:
def compute_and_store_sectors_dfs(years_output, method):
    dfs_dict = dict()
    for sector in years_output[0].keys():
        sector_df = pd.DataFrame(columns = ['years', 'values', 'tickers'])
        for ticker in years_output[0][sector]:
            
            if method in ['DCF', 'EPS', 'NCAVPS', 'GGM']:
                ticker_values_dict = {year:years_output[year][sector][ticker][1] for year in years_output.keys() if ticker in years_output[year][sector].keys()}
            elif method in ['piotroskiFscore', 'ranking']:
                ticker_values_dict = {year:years_output[year][sector][ticker] for year in years_output.keys() if ticker in years_output[year][sector].keys()}
            ticker_df = pd.DataFrame({'years': [-i for i in ticker_values_dict.keys()], 'values': list(ticker_values_dict.values()), 'tickers': list(repeat(ticker, len(ticker_values_dict)))}) 
            sector_df = pd.concat([sector_df, ticker_df])
        dfs_dict[sector]=sector_df
        #to_csv(sector, sector_df, method)
    return dfs_dict

In [16]:
def get_sector(ticker):
    sector = fa.profile(ticker, api_key).loc['sector', 0]
    return sector

In [17]:
def get_ticker_evolution_plot(ticker, method):
    years_output = get_method_files(method, 'year.pickle')
    dfs_dict = compute_and_store_sectors_dfs(years_output, method)
    ticker_to_plot = dfs_dict[get_sector(ticker)].loc[dfs_dict[get_sector(ticker)].tickers == ticker]
    if method == 'piotroskiFscore' and ticker == 'NVDA':
        data = []
        data.insert(0, {'years': -7, 'values': 7, 'tickers': 'NVDA'})
        ticker_to_plot = pd.concat([pd.DataFrame(data), ticker_to_plot], ignore_index=True)
    fig = px.line(ticker_to_plot, 
                  x="years", 
                  y="values",
                  title=f'Line chart of the evolution of the {method} for ticker: {ticker}')
    fig.show()

In [18]:
def get_sector_evolution_plot(sector, method):
    years_output = get_method_files(method, 'year.pickle')
    dfs_dict = compute_and_store_sectors_dfs(years_output, method)
    fig = px.line(dfs_dict[sector], 
                  x="years", 
                  y="values", 
                  color='tickers',
                  title=f'Line chart of the evolution of the {method} values across companies for sector: {sector}')
    fig.show()

--------------
---------------
------------
-------------

#######**ABSOLUTE VALUATION**

**Absolute value specific functions**

In [19]:
def ABS_compute_percentage_winners_per_sector(method):
    years_output = get_method_files(method, 'year.pickle')
    evolution_per_sector_sorted = get_evolution_per_sector_sorted(years_output, method)
    winners_per_sector = dict()
    for sector in evolution_per_sector_sorted.keys():
        winners_per_sector[sector] = dict()
        for year_disp in evolution_per_sector_sorted[sector].keys():
            number_of_winners = sum([1 for stock in evolution_per_sector_sorted[sector][year_disp].values() if stock[1] > 1])
            winners_per_sector[sector][year_disp] = (number_of_winners/len(evolution_per_sector_sorted[sector][year_disp]))*100
    return winners_per_sector

In [20]:
def ABS_compute_average_percentage_winners_per_sector(method):
    percentage_winners_per_sector = ABS_compute_percentage_winners_per_sector(method)
    avg_winners_per_sector = dict()
    for sector in percentage_winners_per_sector.keys():
        avg_winners_per_sector[sector] = mean(list(percentage_winners_per_sector[sector].values()))
    return avg_winners_per_sector

In [21]:
def ABS_compute_average_percentage_winners_per_year(method):
    percentage_winners_per_sector = ABS_compute_percentage_winners_per_sector(method)
    avg_winners_per_year = dict()
    for year_disp in percentage_winners_per_sector['Industrials'].keys():
        sectors_values = []
        for sector in percentage_winners_per_sector.keys():
            sectors_values.append(percentage_winners_per_sector[sector][year_disp])
        avg_winners_per_year[year_disp] = mean(sectors_values)
    return avg_winners_per_year

-------------------------------------------------------#####**DCF**---------------------------------------------------

In [22]:
method = 'DCF'
years_output = get_method_files(method, 'year.pickle')

*Get a (number of sectors) dataframes with columns=['years', 'values', 'tickers'] (where 'values' are the intrinsic values)*

In [ ]:
#Get dfs for each sector (considering all the years)
sectors_dfs = compute_and_store_sectors_dfs(years_output, method)
sectors_dfs['Industrials'].head()

*Get evolution by sector and year sorted decreasingly by the instrinsic values*

In [27]:
evolution_per_sector_sorted = get_evolution_per_sector_sorted(years_output, method)
evolution_per_year_sorted = get_evolution_per_year_sorted(years_output, method)

In [ ]:
evolution_per_sector_sorted

*Get percentage of winners for sector and year*

In [ ]:
percentage_winners_per_sector = ABS_compute_percentage_winners_per_sector(method)
percentage_winners_per_sector

*Stock evolution*

*Visualize evolution for any S&P 500 ticker*

In [ ]:
ticker='NVDA'

In [ ]:
get_ticker_evolution_plot(ticker, method)

**Sector evolution**

Visualize evolution for any sector

In [ ]:
sector = 'Technology'
get_sector_evolution_plot(sector, method)

---------------------------------------------#####**EPS**------------------------------------------------------------

In [ ]:
method = 'EPS'
years_output = get_method_files(method, 'year.pickle')

In [ ]:
years_output.keys()

In [ ]:
evolution_per_sector_sorted = get_evolution_per_sector_sorted(years_output, method)
evolution_per_year_sorted = get_evolution_per_year_sorted(years_output, method)

In [ ]:
ABS_compute_average_percentage_winners_per_sector(method)

In [ ]:
ABS_compute_average_percentage_winners_per_year(method)

In [ ]:
evolution_per_sector_sorted = get_evolution_per_sector_sorted(years_output, method)
for year_disp in evolution_per_sector_sorted['Technology'].keys():
    print(evolution_per_sector_sorted['Technology'][year_disp][ticker])

In [ ]:
get_ticker_evolution_plot(ticker, method)

In [ ]:
sector = 'Technology'
get_sector_evolution_plot(sector, method)

-----------------------------------------------------#####**NCAVPS**--------------------------------------------------

This method results are the most tightly related with the price itself. Maybe it is because the complexity of the method is low and therefore it is probably the case that it is used to identify good opportunities

In [ ]:
method = 'NCAVPS'
years_output = get_method_files(method, 'year.pickle')

In [ ]:
years_output.keys()

In [ ]:
evolution_per_sector_sorted = get_evolution_per_sector_sorted(years_output, method)
evolution_per_year_sorted = get_evolution_per_year_sorted(years_output, method)

In [ ]:
evolution_per_sector_sorted['Technology']

In [ ]:
ABS_compute_average_percentage_winners_per_sector(method)

In [ ]:
ABS_compute_average_percentage_winners_per_year(method)

In [ ]:
evolution_per_sector_sorted = get_evolution_per_sector_sorted(years_output, method)
for year_disp in evolution_per_sector_sorted['Technology'].keys():
    print(evolution_per_sector_sorted['Technology'][year_disp][ticker])

In [ ]:
get_ticker_evolution_plot(ticker, method)

In [ ]:
sector = 'Technology'
get_sector_evolution_plot(sector, method)

------------------------------------------------------------#####**GGM**-------------------------------------------

In [ ]:
method = 'GGM'
years_output = get_method_files(method, 'year.pickle')

In [ ]:
years_output.keys()

In [ ]:
evolution_per_sector_sorted = get_evolution_per_sector_sorted(years_output, method)
evolution_per_year_sorted = get_evolution_per_year_sorted(years_output, method)

In [ ]:
evolution_per_sector_sorted['Technology']

In [ ]:
ABS_compute_average_percentage_winners_per_sector(method)

In [ ]:
ABS_compute_average_percentage_winners_per_year(method)

In [ ]:
get_ticker_evolution_plot(ticker, method)

-----------------
--------------------
---------------
-------------------

#######**RELATIVE VALUATION**

In [ ]:
def REL_compute_percentage_winners_per_sector(method):
    years_output = get_method_files(method, 'year.pickle')
    evolution_per_sector_sorted = get_evolution_per_sector_sorted(years_output, method)
    winners_per_sector = dict()
    if method == 'piotroskiFscore':
        for sector in evolution_per_sector_sorted.keys():
            winners_per_sector[sector] = dict()
            for year_disp in evolution_per_sector_sorted[sector].keys():
                number_of_winners = sum([1 for stock in evolution_per_sector_sorted[sector][year_disp].values() if stock>=8])
                winners_per_sector[sector][year_disp] = (number_of_winners/len(evolution_per_sector_sorted[sector][year_disp]))*100
    else:
        for sector in evolution_per_sector_sorted.keys():
            winners_per_sector[sector] = dict()
            for year_disp in evolution_per_sector_sorted[sector].keys():
                quartile1 = np.quantile(list(evolution_per_sector_sorted[sector][year_disp].values()), [0.25])[0]
                number_of_winners = sum([1 for stock in evolution_per_sector_sorted[sector][year_disp].values() if stock>0 and stock<quartile1])
                winners_per_sector[sector][year_disp] = (number_of_winners/len(evolution_per_sector_sorted[sector][year_disp]))*100
        pass
    return winners_per_sector

In [ ]:
def REL_compute_average_percentage_winners_per_sector(method):
    percentage_winners_per_sector = REL_compute_percentage_winners_per_sector(method)
    avg_winners_per_sector = dict()
    for sector in percentage_winners_per_sector.keys():
        avg_winners_per_sector[sector] = mean(list(percentage_winners_per_sector[sector].values()))
    return avg_winners_per_sector

In [ ]:
def REL_compute_average_percentage_winners_per_year(method):
    percentage_winners_per_sector = REL_compute_percentage_winners_per_sector(method)
    avg_winners_per_year = dict()
    for year_disp in percentage_winners_per_sector['Industrials'].keys():
        sectors_values = []
        for sector in percentage_winners_per_sector.keys():
            sectors_values.append(percentage_winners_per_sector[sector][year_disp])
        avg_winners_per_year[year_disp] = mean(sectors_values)
    return avg_winners_per_year

-------------------------------------------------#####**Piotroski F score**------------------------------------------

In [ ]:
method = 'piotroskiFscore'
years_output = get_method_files(method, 'year.pickle')

In [ ]:
years_output.keys()

In [ ]:
evolution_per_sector_sorted = get_evolution_per_sector_sorted(years_output, method)
evolution_per_year_sorted = get_evolution_per_year_sorted(years_output, method)

In [ ]:
evolution_per_sector_sorted['Technology']

In [ ]:
percentage_winners_per_sector = REL_compute_percentage_winners_per_sector(method)
percentage_winners_per_sector

In [ ]:
REL_compute_average_percentage_winners_per_sector(method)

In [ ]:
REL_compute_average_percentage_winners_per_year(method)

In [ ]:
get_ticker_evolution_plot(ticker, method)

In [ ]:
sector = 'Industrials'
get_sector_evolution_plot(sector, method)

---------

------------------------------------------------#####**Naive ranking**---------------------------------------------

In [ ]:
method = 'ranking'
years_output = get_method_files(method, 'year.pickle')

In [ ]:
years_output.keys()

In [ ]:
evolution_per_sector_sorted = get_evolution_per_sector_sorted(years_output, method)
evolution_per_year_sorted = get_evolution_per_year_sorted(years_output, method)

In [ ]:
REL_compute_average_percentage_winners_per_sector(method)

In [ ]:
REL_compute_average_percentage_winners_per_year(method)

In [ ]:
get_ticker_evolution_plot(ticker, method)

In [ ]:
sector = 'Industrials'
get_sector_evolution_plot(sector, method)